In [3]:

import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate

st.title('Stock Trading Strategy Backtesting App')

# Get user inputs
principal = st.number_input('Enter the principal amount:', value=10000.0)
symbol = st.text_input('Enter the stock symbol:')
start_date = st.date_input('Enter the start date:')
end_date = st.date_input('Enter the end date:')
strategy = st.selectbox('Select the trading strategy:', options=['SMA', 'EMA'])

if st.button('Run Backtest'):
    # Read data for the given stock symbol and date range
    data = pd.read_csv(f"/content/drive/MyDrive/Data/{symbol}.csv", parse_dates=['Date'], index_col='Date')
    data = data.loc[start_date:end_date]

    # Calculate moving averages and trading signals
    if strategy == 'SMA':
        data['20_SMA'] = data.Close.rolling(window=20, min_periods=1).mean()
        data['50_SMA'] = data.Close.rolling(window=50, min_periods=1).mean()
        data['Signal'] = np.where(data['20_SMA'] > data['50_SMA'], 1, 0)
    elif strategy == 'EMA':
        data['20_EMA'] = data.Close.ewm(span=20, adjust=False).mean()
        data['50_EMA'] = data.Close.ewm(span=50, adjust=False).mean()
        data['Signal'] = np.where(data['20_EMA'] > data['50_EMA'], 1, 0)
    else:
        st.warning('Invalid strategy selected!')
        st.stop()

    data['Position'] = data.Signal.diff()

    # Create plot
    fig, ax = plt.subplots(figsize=(20,10))
    data['Close'].plot(color = 'k', label= 'Close Price')
    if strategy == 'SMA':
        data['20_SMA'].plot(color = 'b',label = '20-day SMA')
        data['50_SMA'].plot(color = 'g', label = '50-day SMA')
    elif strategy == 'EMA':
        data['20_EMA'].plot(color = 'b',label = '20-day EMA')
        data['50_EMA'].plot(color = 'g', label = '50-day EMA')
    ax.plot(data[data['Position'] == 1].index, data['Close'][data['Position'] == 1], '^', markersize = 15, color = 'g', label = 'buy')
    ax.plot(data[data['Position'] == -1].index, data['Close'][data['Position'] == -1], 'v', markersize = 15, color = 'r', label = 'sell')
    ax.set_ylabel('Price in Rupees', fontsize = 15 )
    ax.set_xlabel('Date', fontsize = 15 )
    ax.set_title(f'{symbol.upper()} ({strategy.upper()} strategy, Principal = {principal})', fontsize = 20)
    ax.legend()
    ax.grid()
    st.pyplot(fig)

    # Create and print position table
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x==1 else 'Sell')
    df_pos.rename(columns={'Position': 'Action'}, inplace=True)
    df_pos.insert(0, 'Trade Date', df_pos.index)
    df_pos.reset_index(drop=True, inplace=True)
    df_pos['Quantity'] = principal / df_pos['Close']
    df_pos['Amount'] = df_pos['Quantity'] * df_pos['Close']
    df_pos['Cumulative Amount'] = df_pos['Amount'].cumsum()
    st.subheader('Position Table')
    st.table(df_pos[['Trade Date', 'Action', 'Close', 'Quantity', 'Amount', 'Cumulative Amount']].style.format({
    'Close': '{:.2f}',
    'Quantity': '{:.2f}',
    'Amount': '{:.2f}',
    'Cumulative Amount': '{:.2f}',
    }))

    # Print performance metrics
    st.write('## Performance Metrics')
    total_trades = len(df_pos)
    win_trades = len(df_pos[df_pos['Action']=='Sell'])
    loss_trades = total_trades - win_trades
    win_rate = win_trades/total_trades*100
    
    st.write(f'Total Trades: {total_trades}')
    st.write(f'Win Trades: {win_trades}')
    st.write(f'Loss Trades: {loss_trades}')
    st.write(f'Win Rate: {win_rate:.2f}%')



  command:

    streamlit run /usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2023-04-02 05:41:45.973 
  command:

    streamlit run /usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [2]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.1 MB/s eta 0:00:00


In [9]:
!streamlit run my_project5.py & npx localtunnel --port 8501

[##................] | fetchMetadata: sill resolveWithNewModule ansi-styles@4.3


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.77.182:8501

npx: installed 22 in 2.953s
your url is: https://free-zoos-joke-34-125-77-182.loca.lt
2023-04-02 06:22:30.185 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 565, in _run_script
    exec(code, module.__dict__)
  File "/content/my_project5.py", line 93, in <module>
    drive.mount('/content/drive')
  File "/usr/local/lib/python3.9/dist-packages/google/colab/drive.py", line 103, in mount
    return _mount(
  File "/usr/local/lib/python3.9/dist-packages/google/colab/drive.py", line 132, in _mount
    _message.blocking_request(
  File "/usr/local/lib/python3.9/dist-packages/google/colab/_message.py", line 174, in blocking_request
    request_id = send_request(
  F

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section